# Example of an RNN

We construct a simple RNN to generate text from a work of Nietsche

## Setup libraries and functions.

We start by importing various libraries needed.

In [1]:
# Plots displayed inline in notebook
%matplotlib inline

# Make help libraries available
import sys

sys.path.append('D:/anlaursen/libraries')

# Set visible devices, so as to just use a single GPU.
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
import numpy as np

from numpy.random import choice

from keras.models import Sequential
from keras.layers import Embedding, LSTM, TimeDistributed, Activation
from keras.layers.core import Dropout, Dense
from keras.optimizers import Adam
from keras.utils import get_file

from keras import backend as K
K.set_image_data_format('channels_last')

Using TensorFlow backend.


## Setup data

Import the Nietsche data and save it as text.

In [3]:
path = get_file('nietzsche.txt',
                origin = "https://s3.amazonaws.com/text-datasets/nietzsche.txt")
text = open(path).read().lower()

print('\ncorpus length:', len(text))


corpus length: 600901


Let's consider an example. We see that the text is very convoluted (haha), which in this case make it easier for a model, as it will be hard for us humans to undertand wether something is true Nietsche tect.

In [4]:
print(text[-1750:])

 all existence, in an impending day of
judgment. in the last rays of the setting sun of the ancient world,
which fell upon the christian peoples, the shadowy form of the saint
attained enormous proportions--to such enormous proportions, indeed,
that down even to our own age, which no longer believes in god, there
are thinkers who believe in the saints.


144

it stands to reason that this sketch of the saint, made upon the model
of the whole species, can be confronted with many opposing sketches that
would create a more agreeable impression. there are certain exceptions
among the species who distinguish themselves either by especial
gentleness or especial humanity, and perhaps by the strength of their
own personality. others are in the highest degree fascinating because
certain of their delusions shed a particular glow over their whole
being, as is the case with the founder of christianity who took himself
for the only begotten son of god and hence felt himself sinless; so that
through

We then make a list of all charcters used in the text. And determine the size of the vocabulary.

In [5]:
chars = sorted(list(set(text)))
vocab_size = len(chars) + 1
print('total chars:', vocab_size)

total chars: 60


Sometimes it's useful to have a zero value in the dataset, e.g. for padding, so we add that. And consider the output (-6 removes forreign utf-8 characters).

In [6]:
chars.insert(0, "\0")

In [7]:
''.join(chars[1:-6])

'\n !"\'(),-.0123456789:;=?[]_abcdefghijklmnopqrstuvwxyz'

We then make a dict for characters and for indices. So each character gets mapped to an indecy

In [8]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

We then convert the text using these indices.

In [9]:
idx = [char_indices[c] for c in text]

Consider the first ten characters.

In [10]:
idx[:10]

[43, 45, 32, 33, 28, 30, 32, 1, 1, 1]

Corresponding to (Note the annoying line feeds)

In [11]:
''.join(indices_char[i] for i in idx[:10])

'preface\n\n\n'

## Preprocess and create model

We then construct running sentences where each sentence in a pair is one character aahead of the other sentence in the pair.

In [12]:
maxlen = 40
sentences = []
next_chars = []
for i in range(0, len(idx) - maxlen + 1):
    sentences.append(idx[i: i + maxlen])
    next_chars.append(idx[i+1: i + maxlen + 1])
print('nb sequences:', len(sentences))

nb sequences: 600862


We can then construct an array of sentences and the same sentences moved one character forward. Essentially a training set and a label set.

In [13]:
sentences = np.concatenate([[np.array(o)] for o in sentences[:-2]])
next_chars = np.concatenate([[np.array(o)] for o in next_chars[:-2]])

We can validate that the two arrays have the same shape. That is 600860 sentences of 40 characters.

In [14]:
sentences.shape, next_chars.shape

((600860, 40), (600860, 40))

We speccify the number of factors to use in our embedding of the sentences.

In [15]:
n_fac = 24

An we can then contruct out RNN, specifically a long-short term memory recurrent neural network.

In [16]:
model = Sequential([
        Embedding(vocab_size, n_fac, input_length = maxlen),
        LSTM(512,
             input_shape = (None, n_fac),
             return_sequences = True,
             recurrent_dropout = 0.2,
             dropout = 0.2,
             # Implementation: One of {0, 1, or 2}. If set to 0, the RNN will use an
             # implementation that uses fewer, larger matrix products, thus running 
             # faster on CPU but consuming more memory. If set to 1, the RNN will use 
             # more matrix products, but smaller ones, thus running slower (may 
             # actually be faster on GPU) while consuming less memory. If set to 2 
             # (LSTM/GRU only), the RNN will combine the input gate, the forget gate 
             # and the output gate into a single matrix, enabling more time-efficient 
             # parallelization on the GPU.
             implementation = 2),
        Dropout(0.2),
        LSTM(512,
             return_sequences = True,
             recurrent_dropout = 0.2,
             dropout = 0.2,
             implementation = 2),
        Dropout(0.2),
        TimeDistributed(Dense(vocab_size)),
        Activation('softmax')
    ])    

We compile the model

In [17]:
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = Adam())

## Train the LSTM RNN

We define a function such as to print sample predictions.

In [18]:
def print_example():
    
    # WE have to see the model, so we pick something, that sounds Nietsche like.
    seed_string = "ethics is a basic foundation of all that"
    
    for i in range(320):
        x = np.array([char_indices[c] for c in seed_string[-40:]])[np.newaxis, :]
        preds = model.predict(x, verbose = 0)[0][-1]
        preds = preds / np.sum(preds)
        next_char = choice(chars, p = preds)
        seed_string = seed_string + next_char
    print(seed_string)

In [19]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 40, 24)            1440      
_________________________________________________________________
lstm_1 (LSTM)                (None, 40, 512)           1099776   
_________________________________________________________________
dropout_1 (Dropout)          (None, 40, 512)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 40, 512)           2099200   
_________________________________________________________________
dropout_2 (Dropout)          (None, 40, 512)           0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 40, 60)            30780     
_________________________________________________________________
activation_1 (Activation)    (None, 40, 60)            0         
Total para

We then train the model for a single epoch

In [20]:
model.fit(sentences, np.expand_dims(next_chars, -1), batch_size = 64, epochs = 1)

Epoch 1/1
600860/600860 [==============================] - 1302s - loss: 1.5140  


And print the prediction. After the first epoch we see that sentence and word structure is starting to form, but the output is rather terrible.

In [21]:
print_example()

ethics is a basic foundation of all that for his basis and nothing! so around
coor. from nature which is
in called "strength," we have reac and sevented supposing a master than the dome conscience likewise a more
repugiated, the -orrance and dislike, and always, prosure of impossible quite his
head that ruling civilization but he must
be essentially explaine


So we run for an epoch more.

In [22]:
model.fit(sentences, np.expand_dims(next_chars, -1), batch_size = 64, epochs = 1)

Epoch 1/1
600860/600860 [==============================] - 1282s - loss: 1.2868  


Now it's actually starting to construct words, sentences and using punctuation. But it is still rather meaningsless.

In [23]:
print_example()

ethics is a basic foundation of all that account for a
distance, because man who is
essential to generation, that is to say, a "science more to slead," with regard to friends this
formerly disguise, and so luther at all, which an eternal latter--delusions of "man" a sort of raised
man: this church has been interesting fly.--young soul is not only the worst r


So we decrease the learning rate and run for another epoch

In [24]:
model.optimizer.lr = 0.001

model.fit(sentences, np.expand_dims(next_chars, -1), batch_size = 64, epochs = 1)

Epoch 1/1
600860/600860 [==============================] - 1299s - loss: 1.2526  


Now we're starting to get somewhere. We have somthing that can be read and is actual words.

In [25]:
print_example()

ethics is a basic foundation of all that arises
itself out, wull to develop the "pmantom of
our "noble sinkulness" and instincts. (but what happens in the intercourse that he does not possess of the old fellow creature, but thus may be blended that religion and dangerous and enchanted and spirits and also are more refined france of
evil things, for instance,


We decrease the learning rate and run another epoch.

In [26]:
model.optimizer.lr = 0.0001

model.fit(sentences, np.expand_dims(next_chars, -1), batch_size = 64, epochs = 1)

Epoch 1/1
600860/600860 [==============================] - 1292s - loss: 1.2336  


Now we are starting to get chapter marks and paragraph marks as well.

In [27]:
print_example()

ethics is a basic foundation of all that? only these
most successful sentiments so hardly
interpreted about him as the reason also so highly disclosed the
due to be
sure, only in our
issinctions of depth! the man should roluse it in order to have really considered the part of religious havoured, in
the author to make
responsibility and north in religion and 


So we save the weights.

In [28]:
#%mkdir models
model.save_weights('models/char_rnn.h5')

And decrease the learning rate even more and run for another epoch.

In [29]:
model.optimizer.lr = 0.00001

In [30]:
model.fit(sentences, np.expand_dims(next_chars, -1), batch_size = 64, epochs = 1)

Epoch 1/1
    64/600860 [..............................] - ETA: 1549s - loss: 1.2666

C:\Users\anlaursen\AppData\Loca\Anaconda3\envs\py3\lib\site-packages\keras\models.py:844: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


600860/600860 [==============================] - 1299s - loss: 1.2190  


Now we are getting something quite nietsche like.

In [31]:
print_example()

ethics is a basic foundation of all that are blame they
rank and at the same time in its ultruth; and
out require their regularity
of the occasions).--he writes to himself about conquering fundamental boldness
of his learned men, if a philosopher, to possession to
reuden, is such as conduct. but if it were that
it is not a delicate, to look from
which the ri


Let's run a final epoch and consider two examples. Very Nietsche like now.

In [33]:
model.fit(sentences, np.expand_dims(next_chars, -1), batch_size = 64, epochs = 1)

Epoch 1/1
600860/600860 [==============================] - 1294s - loss: 1.2015  


In [34]:
print_example()

ethics is a basic foundation of all that an act of a
danger in a finer sensation of the condemned man, humanity has given before the "happiness" extend)
could not nover indirectly be greater into
aul events can be frigntful, although he will come to him and agony the uplers of system of
tragedy--which a means, only confounds a second, moral,
sacrifice, goal



In [35]:
print_example()

ethics is a basic foundation of all that is transfigurated;--and potent woman renounces over-rich
more the striving for restraint through everything draws at
the whole spirit man grows
pleasure and deny myn, divined whatever, around
whatever they should have honoured by his nature.


123

=stupidity--the called "will," they wish
to be slain own experiences w


Now it can also generate proper chapter marks.

In [36]:
model.save_weights('models/char_rnn.h5')